# Paprastas vaizdų klasifikatorius

Šiame užrašų knygelėje parodoma, kaip klasifikuoti vaizdus naudojant iš anksto apmokytą neuroninį tinklą.

**Ką išmoksite:**
- Kaip įkelti ir naudoti iš anksto apmokytą modelį
- Vaizdų išankstinis apdorojimas
- Prognozių atlikimas vaizdams
- Pasitikėjimo balų supratimas

**Naudojimo atvejis:** Objektų atpažinimas vaizduose (pvz., „katė“, „šuo“, „automobilis“ ir kt.)

---


## 1 žingsnis: Importuokite reikalingas bibliotekas

Importuokime mums reikalingus įrankius. Nesijaudinkite, jei dar ne viską suprantate!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## 2 žingsnis: Įkelkite iš anksto apmokytą modelį

Naudosime **MobileNetV2**, neuroninį tinklą, kuris jau apmokytas milijonais vaizdų.

Tai vadinama **Perkėlimo mokymusi** – naudojame modelį, kurį apmokė kažkas kitas!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## 3 žingsnis: Pagalbinės funkcijos

Sukurkime funkcijas, skirtas įkelti ir paruošti vaizdus mūsų modeliui.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## 4 žingsnis: Testavimas su pavyzdinėmis nuotraukomis

Išbandykime klasifikavimą su keliomis nuotraukomis iš interneto!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Klasifikuokite Kiekvieną Vaizdą


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## 5 žingsnis: Išbandykite savo vaizdus!

Pakeiskite žemiau esantį URL bet kokiu vaizdo URL, kurį norite klasifikuoti.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Kas ką tik įvyko?

1. **Įkėlėme iš anksto apmokytą modelį** – MobileNetV2 buvo apmokytas naudojant milijonus vaizdų
2. **Apdorojome vaizdus** – Pakeitėme jų dydį ir formatą, kad jie tiktų modeliui
3. **Modelis atliko prognozes** – Jis pateikė tikimybes 1000 objektų kategorijoms
4. **Dekodavome rezultatus** – Skaičius pavertėme į žmonėms suprantamas etiketes

### Supratimas apie pasitikėjimo balus

- **90–100%**: Labai užtikrintas (beveik tikrai teisinga)
- **70–90%**: Užtikrintas (tikriausiai teisinga)
- **50–70%**: Šiek tiek užtikrintas (gali būti teisinga)
- **Žemiau 50%**: Nelabai užtikrintas (neaišku)

### Kodėl prognozės gali būti neteisingos?

- **Neįprastas kampas ar apšvietimas** – Modelis buvo apmokytas naudojant įprastas nuotraukas
- **Daug objektų** – Modelis tikisi vieno pagrindinio objekto
- **Retai pasitaikantys objektai** – Modelis atpažįsta tik 1000 kategorijų
- **Prastos kokybės vaizdas** – Neryškūs ar pikseliuoti vaizdai yra sudėtingesni


## 🚀 Kiti žingsniai

1. **Išbandykite skirtingus vaizdus:**
   - Raskite vaizdų [Unsplash](https://unsplash.com)
   - Dešiniuoju pelės mygtuku → „Kopijuoti vaizdo adresą“, kad gautumėte URL

2. **Eksperimentuokite:**
   - Kas nutinka su abstrakčiu menu?
   - Ar gali atpažinti objektus iš skirtingų kampų?
   - Kaip jis tvarkosi su keliais objektais?

3. **Sužinokite daugiau:**
   - Naršykite [Kompiuterinės regos pamokas](../lessons/4-ComputerVision/README.md)
   - Išmokite treniruoti savo vaizdų klasifikatorių
   - Supraskite, kaip veikia CNN (Konvoliuciniai neuroniniai tinklai)

---

## 🎉 Sveikiname!

Jūs ką tik sukūrėte vaizdų klasifikatorių naudodami pažangų neuroninį tinklą!

Ši technika naudojama:
- Google Photos (nuotraukų organizavimui)
- Savarankiškai važiuojančiuose automobiliuose (objektų atpažinimui)
- Medicininėje diagnostikoje (rentgeno nuotraukų analizėje)
- Kokybės kontrolėje (defektų aptikimui)

Tęskite tyrinėjimus ir mokymąsi! 🚀



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, atkreipiame dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudotis profesionalių vertėjų paslaugomis. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus aiškinimus, kylančius dėl šio vertimo naudojimo.
